# Sistema de Recomendação Híbrido

Este documento detalha o funcionamento de um script de sistema de recomendação de filmes, construído em Python. O objetivo é criar um modelo robusto que combine duas técnicas principais: **Filtragem Colaborativa** e **Filtragem Baseada em Conteúdo**, resultando em um **Modelo Híbrido**.


In [1]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# --- Carregamento e Preparação dos Dados ---

# Carregar os datasets que você processou
ratings_df = pd.read_csv(r'../data/data_output/ratings_processed.csv')
movies_df = pd.read_csv(r'../data/data_output/movies_processed.csv')

# A biblioteca surprise precisa que definamos a escala das avaliações (1 a 5)
reader = Reader(rating_scale=(0.5, 5))

# Carregar os dados a partir do dataframe do pandas
# Usamos apenas as colunas essenciais para a filtragem colaborativa: user, item, rating
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# --- Divisão em Treino e Teste ---

# Dividir os dados em 75% para treino e 25% para teste
# O random_state garante que a divisão seja sempre a mesma, para reprodutibilidade
print("Dividindo os dados em conjuntos de treino e teste...")
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

print("Dados divididos com sucesso!")

# --- Treinamento do Modelo SVD ---

# Instanciar o algoritmo SVD
# n_factors: O número de fatores latentes (um bom ponto de partida é 100)
# n_epochs: O número de iterações do algoritmo sobre os dados (20 é um valor comum)
# random_state: Para garantir resultados reprodutíveis
print("Treinando o modelo SVD...")
algo = SVD(n_factors=100, n_epochs=20, random_state=42)

# Treinar o algoritmo com o conjunto de treino
algo.fit(trainset)

print("Modelo treinado com sucesso!")

Dividindo os dados em conjuntos de treino e teste...
Dados divididos com sucesso!
Treinando o modelo SVD...
Modelo treinado com sucesso!


In [3]:
# --- Avaliação do Modelo ---

# Fazer previsões no conjunto de teste
print("Avaliando o modelo com o conjunto de teste...")
predictions = algo.test(testset)

# Calcular o RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)

print(f"O RMSE do modelo no conjunto de teste é: {rmse}")

Avaliando o modelo com o conjunto de teste...
RMSE: 0.8820
O RMSE do modelo no conjunto de teste é: 0.8820442070964672


In [4]:
# Calcular o MAE (Mean Absolute Error)

print("Avaliando MAE (Mean Absolute Error)...")
mae = accuracy.mae(predictions)

print(f"O MAE do modelo no conjunto de teste é: {mae}")

Avaliando MAE (Mean Absolute Error)...
MAE:  0.6784
O MAE do modelo no conjunto de teste é: 0.6784083137727962


In [5]:
def get_top_n_recommendations(user_id, n=10):
    """
    Gera as N melhores recomendações para um usuário específico.
    """
    # 1. Obter a lista de todos os IDs de filmes que o usuário JÁ avaliou.
    movies_rated_by_user = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
    
    # 2. Obter a lista de TODOS os IDs de filmes.
    all_movie_ids = movies_df['movieId'].unique()
    
    # 3. Obter a lista de filmes que o usuário NÃO avaliou.
    movies_to_predict = [movie_id for movie_id in all_movie_ids if movie_id not in movies_rated_by_user]
    
    # 4. Dicionário para armazenar as previsões
    predictions_dict = {}
    
    # 5. Prever a avaliação para cada filme não avaliado
    for movie_id in movies_to_predict:
        # A função .predict() retorna a previsão da avaliação
        prediction = algo.predict(uid=user_id, iid=movie_id)
        predictions_dict[movie_id] = prediction.est # 'est' é a nota estimada
        
    # 6. Ordenar as previsões da maior para a menor
    sorted_predictions = sorted(predictions_dict.items(), key=lambda item: item[1], reverse=True)
    
    # 7. Obter os IDs dos N melhores filmes recomendados
    top_n_movie_ids = [movie_id for movie_id, rating in sorted_predictions[:n]]
    
    # 8. Obter os títulos dos filmes recomendados
    recommended_movies = movies_df[movies_df['movieId'].isin(top_n_movie_ids)][['movieId', 'title', 'genres']]
    
    return recommended_movies

In [6]:
# --- Exemplo de Uso ---
# Vamos gerar recomendações para o usuário de ID 50
user_example_id = 50
top_10_recommendations = get_top_n_recommendations(user_id=user_example_id, n=10)

print(f"\\n--- Top 10 Recomendações para o Usuário {user_example_id} ---")
print(top_10_recommendations)

\n--- Top 10 Recomendações para o Usuário 50 ---
      movieId                                              title  \
650       838                                        Emma (1996)   
686       904                                 Rear Window (1954)   
692       910                            Some Like It Hot (1959)   
935      1235                            Harold and Maude (1971)   
943      1244                                   Manhattan (1979)   
2020     2692                   Run Lola Run (Lola rennt) (1998)   
2765     3703               Road Warrior, The (Mad Max 2) (1981)   
3622     4973  Amelie (Fabuleux destin d'Amélie Poulain, Le) ...   
4909     7361       Eternal Sunshine of the Spotless Mind (2004)   
6315    48516                               Departed, The (2006)   

                                genres  
650               Comedy|Drama|Romance  
686                   Mystery|Thriller  
692                       Comedy|Crime  
935               Comedy|Drama|Romance

In [7]:
def precision_recall_at_k(predictions, k=10, threshold=4.0):
    """
    Retorna a precision e recall médios em k para todos os usuários.
    """

    # Mapear as previsões para cada usuário.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Ordenar as previsões do usuário da maior para a menor nota estimada
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Número de itens recomendados no top-k
        n_rec_k = k
        
        # Número de itens relevantes (que o usuário realmente gostou, ex: nota >= 4.0)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Número de itens relevantes E recomendados no top-k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Cálculo da Precision@k
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Cálculo da Recall@k
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    # Média da precisão e recall de todos os usuários
    return sum(p for p in precisions.values()) / len(precisions), \
           sum(r for r in recalls.values()) / len(recalls)

In [8]:
# (Após o cálculo do RMSE e MAE)
print("\nAvaliando Precision@k e Recall@k...")
k_val = 10
rating_threshold = 4.0  # Definimos que "gostar" é dar nota 4.0 ou mais
precision, recall = precision_recall_at_k(predictions, k=k_val, threshold=rating_threshold)

print(f"Precision@{k_val} (limite da nota: {rating_threshold}): {precision:.4f}")
print(f"Recall@{k_val} (limite da nota: {rating_threshold}): {recall:.4f}")


Avaliando Precision@k e Recall@k...
Precision@10 (limite da nota: 4.0): 0.3651
Recall@10 (limite da nota: 4.0): 0.2787


In [9]:
# 1. Preparar a coluna de géneros (substituir '|' por ' ')
# (Vamos verificar se 'title_clean' já existe, caso contrário, pegamos 'title')
if 'title_clean' not in movies_df.columns:
    movies_df['title_clean'] = movies_df['title'].str.replace(r' \(\d{4}\)$', '', regex=True).str.strip()
    
movies_df['genres_space_separated'] = movies_df['genres'].str.replace('|', ' ', regex=False)

# 2. Criar a Matriz TF-IDF
print("Criando matriz TF-IDF...")
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres_space_separated'])

# 3. Calcular a Matriz de Similaridade de Cossenos
print("Calculando matriz de similaridade...")
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 4. Criar o mapa de índices (título -> índice)
indices = pd.Series(movies_df.index, index=movies_df['title_clean'])
indices = indices[~indices.index.duplicated(keep='first')]

Criando matriz TF-IDF...
Calculando matriz de similaridade...


In [10]:
# --- Fase 2 (Continuação): Criando a Função de Recomendação de Conteúdo ---

def get_content_based_recommendations(title, cosine_sim=cosine_sim, data=movies_df, indices=indices, n=10):
    """
    Gera as N melhores recomendações baseadas em conteúdo (géneros).
    """
    try:
        # 1. Obter o índice do filme que corresponde ao título
        idx = indices[title]

        # 2. Obter as pontuações de similaridade de todos os filmes com esse filme
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 3. Ordenar os filmes com base nas pontuações de similaridade
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4. Obter as pontuações dos 10 filmes mais similares (ignorando o primeiro, que é o próprio filme)
        sim_scores = sim_scores[1:n+1]

        # 5. Obter os índices desses filmes
        movie_indices = [i[0] for i in sim_scores]

        # 6. Retornar os títulos dos filmes mais similares
        return data['title_clean'].iloc[movie_indices]
    
    except KeyError:
        return f"Erro: Filme '{title}' não encontrado no banco de dados."

In [11]:
# Escolha um filme que você conhece do 'movies_df'

# Exemplo 1: Recomendações para 'Toy Story'
print("Recomendações baseadas em 'Toy Story':")
print(get_content_based_recommendations('Toy Story'))

print("\n")

# Exemplo 2: Recomendações para 'Heat'
print("Recomendações baseadas em 'Heat':")
print(get_content_based_recommendations('Heat'))

Recomendações baseadas em 'Toy Story':
1706                                                Antz
2355                                         Toy Story 2
2809             Adventures Of Rocky And Bullwinkle, The
3000                           Emperor'S New Groove, The
3568                                      Monsters, Inc.
6194                                           Wild, The
6486                                     Shrek The Third
6948                             Tale Of Despereaux, The
7760    Asterix And The Vikings (Astérix Et Les Vikings)
8219                                               Turbo
Name: title_clean, dtype: object


Recomendações baseadas em 'Heat':
22                       Assassins
138     Die Hard: With A Vengeance
156                       Net, The
249           Natural Born Killers
417                 Judgment Night
509                         Batman
793                       Die Hard
1306                     Hard Rain
1315      Replacement Killers, The
1325   

In [12]:
# --- Treinando o Modelo SVD Otimizado Definitivo ---

# 1. Pegar os melhores parâmetros encontrados pelo GridSearch
# (Confirme se estes são os valores que o seu GridSearch retornou)
best_params = {
    'n_epochs': 30,
    'lr_all': 0.01,
    'reg_all': 0.1
}

# 2. Criar o algoritmo final com esses parâmetros
algo_svd_otimizado = SVD(
    n_epochs=best_params['n_epochs'],
    lr_all=best_params['lr_all'],
    reg_all=best_params['reg_all']
)

# 3. Treinar o modelo no CONJUNTO DE DADOS COMPLETO
# (O .build_full_trainset() usa todos os dados, para o modelo aprender o máximo possível)
print("Treinando o modelo SVD Otimizado com todos os dados...")
trainset_full = data.build_full_trainset()
algo_svd_otimizado.fit(trainset_full)

print("Modelo SVD Otimizado treinado com sucesso!")

Treinando o modelo SVD Otimizado com todos os dados...
Modelo SVD Otimizado treinado com sucesso!


In [13]:
def get_hybrid_recommendations(user_id, n=10):
    """
    Gera recomendações híbridas (SVD + Conteúdo) para um usuário.
    """
    
    # --- 1. Obter o filme favorito do usuário (baseado na maior nota) ---
    try:
        user_ratings = ratings_df[ratings_df['userId'] == user_id]
        favorite_movie_id = user_ratings.loc[user_ratings['rating'].idxmax()]['movieId']
        favorite_movie_title = movies_df.loc[movies_df['movieId'] == favorite_movie_id]['title_clean'].iloc[0]
        
        # Obter o índice do filme favorito para usar no 'cosine_sim'
        idx_favorito = indices[favorite_movie_title]
        # Pegar o vetor de similaridade do filme favorito
        sim_scores_favorito = cosine_sim[idx_favorito]
        
    except (ValueError, KeyError):
        # Se o usuário é novo ou o filme não for encontrado, não podemos usar o boost de conteúdo
        print(f"Não foi possível encontrar um filme favorito para o usuário {user_id}. Usando SVD puro.")
        sim_scores_favorito = None

    
    # --- 2. Gerar Candidatos (SVD) ---
    # Obter a lista de filmes que o usuário NÃO avaliou
    movies_rated = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
    all_movie_ids = movies_df['movieId'].unique()
    movies_to_predict = [mid for mid in all_movie_ids if mid not in movies_rated]
    
    svd_predictions = []
    for movie_id in movies_to_predict:
        pred = algo_svd_otimizado.predict(uid=user_id, iid=movie_id)
        svd_predictions.append((movie_id, pred.est))
        
    # Ordenar pelos que o SVD acha que o usuário mais vai gostar
    svd_predictions.sort(key=lambda x: x[1], reverse=True)
    
    # --- 3. Re-ranking (Híbrido) ---
    hybrid_scores = []
    
    # Vamos olhar para os 100 melhores candidatos do SVD
    for movie_id, svd_score in svd_predictions[:100]:
        
        hybrid_score = svd_score  # O score base é a previsão do SVD
        
        if sim_scores_favorito is not None:
            try:
                # Encontrar o índice deste filme candidato
                movie_title = movies_df.loc[movies_df['movieId'] == movie_id]['title_clean'].iloc[0]
                idx_candidato = indices[movie_title]
                
                # Obter a similaridade de conteúdo com o filme favorito
                content_similarity = sim_scores_favorito[idx_candidato]
                
                # Adicionar o "boost" de conteúdo.
                # O '0.5' é um peso que podemos ajustar.
                hybrid_score = svd_score + (content_similarity * 0.5) 
                
            except (KeyError, IndexError):
                pass # Ignora se o filme não for encontrado no mapa de índices
                
        hybrid_scores.append((movie_id, hybrid_score))
        
    # --- 4. Finalizar ---
    # Reordenar a lista com base no score híbrido
    hybrid_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Obter os IDs dos N melhores
    top_n_movie_ids = [mid for mid, score in hybrid_scores[:n]]
    
    # Retornar os títulos
    return movies_df[movies_df['movieId'].isin(top_n_movie_ids)][['movieId', 'title_clean', 'genres']]

In [ ]:
# --- Exemplo de Uso do Modelo Híbrido ---
# Vamos testar com o mesmo usuário 50

print(f"\\n--- Top 10 Recomendações HÍBRIDAS para o Usuário {user_example_id} ---")
print(get_hybrid_recommendations(user_example_id, n=10))

print("\n")

# Vamos comparar com as recomendações originais do SVD (Fase 1)
print(f"\\n--- (Comparação) Top 10 Recomendações SVD Puro para o Usuário {user_example_id} ---")
print(top_10_recommendations) # A variável que guardámos no início

\n--- Top 10 Recomendações HÍBRIDAS para o Usuário 50 ---
      movieId                                     title_clean  \
224       260              Star Wars: Episode IV - A New Hope   
796      1041                                  Secrets & Lies   
841      1104                       Streetcar Named Desire, A   
898      1196  Star Wars: Episode V - The Empire Strikes Back   
2582     3451                    Guess Who'S Coming To Dinner   
4396     6460                         Trial, The (Procès, Le)   
4909     7361           Eternal Sunshine Of The Spotless Mind   
5202     8477                                       Jetée, La   
8301   106642                          Day Of The Doctor, The   
9463   168252                                           Logan   

                       genres  
224   Action|Adventure|Sci-Fi  
796                     Drama  
841                     Drama  
898   Action|Adventure|Sci-Fi  
2582                    Drama  
4396                    Drama  
49

: 